### The second cell of this notebook contains the code of my best agent in the leaderboard(1445.6) and rest cells are just some other of my agents and the code to compare them.
### I have explained the strategy [here in the forum](https://www.kaggle.com/c/santa-2020/discussion/217537).

### If you find it useful upvote is appreciated!

In [ ]:
!pip install kaggle-environments --upgrade
from kaggle_environments import make, evaluate
env = make("mab", debug=True)

In [ ]:
%%writefile myBestAgentInLB.py
import numpy as np

decay_rate = 0.97
n_rounds = 2000
bandit_count = 100
last_reward = 0
total_reward = None 
last_bandit = None
his_hits = None
his_record = None
my_record = None
my_hits = None
wins = None
losses = None
bandits_record = None
record_index = None
bandit_last_step = None

max_depth = 2000
num_probabs = 1001
k_decay1 = np.zeros([max_depth, num_probabs], 'float128')
k_decay0 = np.zeros([max_depth, num_probabs], 'float128')
probabs = np.linspace(0, 1, num_probabs)
for i in range(k_decay1.shape[0]):
    k_decay1[i] = probabs * 0.97 ** i
k_decay0 = 1 - k_decay1

def estimate_probab(seq):
    raw_probab1 = k_decay1[:seq.shape[0],:][seq == 1,:]
    raw_probab0 = k_decay0[:seq.shape[0],:][seq == 0,:]
    each_probab =  raw_probab0.prod(axis = 0) * raw_probab1.prod(axis = 0)
    half_area = each_probab.sum()/ 2
    cum_sum = np.cumsum(each_probab)
    best_estimation =  np.argmin(np.abs(cum_sum - half_area))
    return (probabs[best_estimation]  *.97**seq.shape[0])


myProbabs = None
his_bandit_rec = None
his_rec_index = None
n_lookback = 60
max_hit = 66
probab_ratio = 1.5
n_decision = 7
probab_step = [.47, .40, .35, .31, .26, .24, .21, .175, .15, .13]
avg_proabs = [ .5 * .97**(i* 0.02) for i in range(2000)]
nu_my_bests = 70
nu_my_best_best = 10
my_bests_index = 0
choicePath = None
def new_bandit(step):
    global myProbabs, my_bests_index, choicePath
    start = step - n_lookback if step >= n_lookback else 0
    his_last_moves = his_record[start:step]
    his_last_bandit = his_last_moves[-1]
    n_his_last_move = (his_last_moves == his_last_bandit).sum()
    myProbabs[last_bandit] = estimate_probab(bandits_record[last_bandit,:record_index[last_bandit]])
    myProbabs[his_last_bandit] = estimate_probab(bandits_record[his_last_bandit,: record_index[his_last_bandit]])

    # print(his_last_bandit)
    # print('band rec',bandits_record[his_last_bandit,: record_index[his_last_bandit]])
    # print(his_last_moves)
    # print(n_his_last_move)
    # print('------- Mine:')
    # print(last_bandit)
    # print('band rec:',bandits_record[last_bandit,:record_index[last_bandit]])
    # print(myProbabs)
    not_chosen = (my_hits == 0)
    not_chosen_any = not_chosen.any()

    if step == 1 :  choicePath = {'m1':0, 'm2':0, 'm3':0, 'm4':0,'m5':0, 'm6':0,}
    # if step == 1999 : print(choicePath)
    if  his_hits[his_last_bandit] > 1:
        if my_hits[his_last_bandit] <= 1:
            choicePath['m1'] += 1
            return his_last_bandit
    

    if  not not_chosen_any :
        if n_his_last_move > 1 and my_hits[his_last_bandit] <= 4:
            choicePath['m2'] += 1
            return his_last_bandit


    my_bests = np.argsort(myProbabs)[-nu_my_bests:]
    
    if his_last_bandit in my_bests:
        if my_hits[his_last_bandit] > 0:
            choicePath['m3'] += 1
            return his_last_bandit
    #if np.random.rand() > .5 :
    if not_chosen_any:
        if ((not_chosen) & (his_hits==0)).any():
            choicePath['m4'] += 1
            return np.random.choice(np.where((not_chosen) & (his_hits==0))[0])
        else:
            choicePath['m5'] += 1
            return np.random.choice(np.where(not_chosen)[0])

    # winner = my_bests[-(my_bests_index % nu_my_best_best)-1]
    #winner = np.random.choice( my_bests[-nu_my_best_best:])
    my_best_bests = my_bests[-nu_my_best_best:]
    steps = np.zeros(nu_my_best_best)
    for i in range( nu_my_best_best):
        steps[i] = bandit_last_step[my_best_bests[i]]
    winner = my_best_bests[np.argmin(steps)]
    choicePath['m6'] += 1
    return winner

def agent(obs, conf):
    global bandits_record, my_record, my_hits, his_hits, his_record, myProbabs, his_scores,last_reward
    global last_bandit, total_reward,  record_index, wins, losses, his_rec_index, his_bandit_rec,bandit_last_step
    if obs.step == 0:        
        total_reward = 0 
        his_record = np.zeros(n_rounds, 'int')
        his_hits = np.zeros(conf.banditCount, 'int')
        his_scores = np.zeros(bandit_count) + 0.5
        his_bandit_rec = np.ones([bandit_count, 2000], 'int') * -1
        his_rec_index = np.zeros(bandit_count, 'int')
        
        myProbabs = np.random.rand(bandit_count)* 0.001 + 0.5
        my_record = np.zeros(n_rounds, 'int')
        my_hits = np.zeros(conf.banditCount, 'int')
        bandits_record = np.zeros([conf.banditCount, 2000], 'int')
        record_index = np.zeros(conf.banditCount,'int')
        wins = np.zeros(conf.banditCount, 'int')
        losses = np.zeros(conf.banditCount, 'int')
        bandit_last_step = np.zeros(conf.banditCount, 'int')

        bandit = np.random.randint(conf.banditCount)
        last_bandit = bandit
        return bandit
    if obs.lastActions[0] == last_bandit:
        his_action = obs.lastActions[1]
    else:
        his_action = obs.lastActions[0]
    his_record[obs.step-1] = his_action
    his_hits[his_action] += 1
    his_bandit_rec[his_action,his_rec_index[his_action]] = obs.step
    his_rec_index[his_action] +=1
    bandits_record[his_action, record_index[his_action]] = -1
    record_index[his_action] += 1
    my_hits[last_bandit] += 1
    my_record[obs.step-1] = last_bandit
    if obs.reward > total_reward:
        total_reward += 1
        bandits_record[last_bandit, record_index[last_bandit]] = 1
        wins[last_bandit] += 1
        last_reward = 1
    else:
        bandits_record[last_bandit, record_index[last_bandit]] = 0
        losses[last_bandit] +=1
        last_reward =0
    record_index[last_bandit] += 1
    bandit = int(new_bandit(obs.step))
    last_bandit = bandit 
    bandit_last_step[bandit] = obs.step
    return bandit

In [ ]:
%%writefile random.py
import random
def agent(obs, conf):
    return random.randrange(conf.banditCount)

In [ ]:
%%writefile uniform.py
import random
def agent(obs, conf):
    return obs.step % conf.banditCount

In [ ]:
%%writefile ucb_agent.py
import math

last_bandit = -1
total_reward = 0

sums_of_reward = None
numbers_of_selections = None

def ucb_agent(observation, configuration):    
    global sums_of_reward, numbers_of_selections, last_bandit, total_reward

    if observation.step == 0:
        numbers_of_selections = [0] * configuration["banditCount"]
        sums_of_reward = [0] * configuration["banditCount"]

    if last_bandit > -1:
        reward = observation.reward - total_reward
        sums_of_reward[last_bandit] += reward
        total_reward += reward

    bandit = 0
    max_upper_bound = 0
    for i in range(0, configuration.banditCount):
        if (numbers_of_selections[i] > 0):
            average_reward = sums_of_reward[i] / numbers_of_selections[i]
            delta_i = math.sqrt(2 * math.log(observation.step+1) / numbers_of_selections[i])
            upper_bound = average_reward + delta_i
        else:
            upper_bound = 1e400
        if upper_bound > max_upper_bound and last_bandit != i:
            max_upper_bound = upper_bound
            bandit = i
            last_bandit = bandit

    numbers_of_selections[bandit] += 1

    if bandit is None:
        bandit = 0

    return bandit

In [ ]:
%%writefile probabiliticRounds.py
import random

import numpy as np
sp = np.linspace(0,1,1000)
spp = 1 - sp
decay_rate = 0.97

def probab(n_ones,n_zeros):
    global sp, spp
    cdfBeta = np.cumsum(sp**n_ones * spp **n_zeros )
    place = np.argmin(np.abs(cdfBeta - cdfBeta[999] / 2 ))
    return sp[place]
def probab_with_decay(n_ones,n_zeros,n_opponent):
    global sp, spp, decay_rate
    cdfBeta = np.cumsum(sp**n_ones * spp **n_zeros )
    place = np.argmin(np.abs(cdfBeta - cdfBeta[999] / 2 ))
    return sp[place] * decay_rate**(n_ones + n_zeros + n_opponent)
def probab_decayed(n_ones, n_zeros, n_opponent):
    global sp, spp
    k_decay = n_ones + n_zeros + n_opponent 
    length = int(decay_rate**k_decay * 1000)
    cdfBeta = np.cumsum(sp[:length]**n_ones * spp[:length]**n_zeros )
    place = np.argmin(np.abs(cdfBeta - cdfBeta[length-1] / 2 ))
    return sp[place]

bandit_count = 100
last_result =0
total_reward = 0
last_bandit = None
last_reward = None
my_win_record = np.zeros(bandit_count)
my_loss_record = np.zeros(bandit_count)
my_chances = np.ones(bandit_count) * 0.5
his_overall_record = np.zeros(bandit_count)
total_steps = 2000

def new_bandit(step):
    global my_chances, my_loss_record, my_win_record, his_overall_record
    global last_bandit, total_reward, last_result, last_reward, decay_rate
    progress = step/total_steps
    #relative_decay = 1 - (1 - decay_rate) * progress**0.25
    #decays = decay_rate**(my_win_record + my_loss_record + his_overall_record)
    scores = my_chances * 1 #* decays
    #scores[last_bandit] += (last_reward - .1) * ((1- progress)**0.5)
    scores[last_bandit] += ((last_reward) * ((total_steps - step)/total_steps)**0.5) 
    winner = np.argmax(scores)
    return int(winner)

def agent(obs, conf):
    global my_chances, my_loss_record, my_win_record, his_overall_record
    global last_bandit, total_reward, last_result, last_reward
    if obs.step == 0:        
        bandit = 0
        last_bandit = bandit
        return bandit
    if obs.lastActions[0] == last_bandit:
        his_action = obs.lastActions[1]
    else:
        his_action = obs.lastActions[0]
    his_overall_record[his_action] += 1
            
    if obs.reward > total_reward:
        total_reward += 1
        last_reward = 1
        my_win_record[last_bandit] += 1                        
    else:        
        last_reward = 0
        my_loss_record[last_bandit] += 1
    #my_chances[last_bandit] = my_win_record[last_bandit] / (my_win_record[last_bandit] + my_loss_record[last_bandit])
    #my_chances[last_bandit] = probab(my_win_record[last_bandit], my_loss_record[last_bandit])
    my_chances[last_bandit] = probab_decayed(my_win_record[last_bandit], my_loss_record[last_bandit],his_overall_record[last_bandit])
    my_chances[his_action] = probab_decayed(my_win_record[his_action], my_loss_record[his_action],his_overall_record[his_action])
    bandit = new_bandit(obs.step)
    last_bandit = bandit 
    return bandit

In [ ]:
%%writefile greedyMemory.py
import random

last_result =0
total_reward = 0
last_bandit = None
my_win_record = None
my_loss_record = None
my_chances = None
def agent(obs, conf):
    global my_chances, my_loss_record, my_win_record
    global last_bandit, total_reward,last_result
    if obs.step == 0:        
        my_win_record = [8] * conf.banditCount
        my_loss_record = [1] * conf.banditCount
        my_chances = [9] * conf.banditCount
        bandit = random.randrange(conf.banditCount)
        last_bandit = bandit
        return bandit
    if obs.reward > total_reward:
        total_reward += 1
        my_win_record[last_bandit] += 1
        my_chances[last_bandit] = my_win_record[last_bandit] / (my_win_record[last_bandit] + my_loss_record[last_bandit])
        bandit = last_bandit        
    else:
        my_loss_record[last_bandit] += 1
        my_chances[last_bandit] = my_win_record[last_bandit] / (my_win_record[last_bandit] + my_loss_record[last_bandit])
        bandit = my_chances.index(max(my_chances))        
        last_bandit = bandit
    #print(my_chances)
    #print(bandit)
    return bandit

In [ ]:
%%writefile recentEvents.py
import numpy as np

decay_rate = 0.97

def probab(n_ones,n_zeros):
    global sp, spp
    cdfBeta = np.cumsum(sp**n_ones * spp **n_zeros )
    place = np.argmin(np.abs(cdfBeta - cdfBeta[999] / 2 ))
    return sp[place]

bandit_count = 100
total_reward = 0
last_bandit = None
his_overall_record = np.zeros(bandit_count)
total_steps = 2000

record_size = 7
bandits_record = np.zeros([record_size,bandit_count])
record_index = np.zeros(bandit_count,'int')
my_hits_p_1 = np.ones(bandit_count)

def new_bandit(step):
    global bandits_record, his_overall_record, last_bandit 
    exploration_score = 1 / my_hits_p_1
    scores = bandits_record.sum(axis=0) + 2.5 * exploration_score
    winner = np.argmax(scores)    
    return int(winner)

def agent(obs, conf):
    global bandits_record, his_overall_record
    global last_bandit, total_reward
    if obs.step == 0:        
        bandit = np.random.randint(bandit_count)
        last_bandit = bandit
        return bandit
    if obs.lastActions[0] == last_bandit:
        his_action = obs.lastActions[1]
    else:
        his_action = obs.lastActions[0]
    his_overall_record[his_action] += 1
    my_hits_p_1[last_bandit] += 1
    if obs.reward > total_reward:
        total_reward += 1
        bandits_record[record_index[last_bandit], last_bandit] = 1        
    else:
        bandits_record[record_index[last_bandit], last_bandit] = -1
    record_index[last_bandit] = (record_index[last_bandit] + 1) % record_size 
    bandit = new_bandit(obs.step)
    last_bandit = bandit 
    return bandit

In [ ]:
%%writefile greedyMemoryWithExpolaration.py
import random

last_result =0
total_reward = 0
last_bandit = None
my_win_record = None
my_loss_record = None
my_chances = None
chances = None
def agent(obs, conf):
    global my_chances, my_loss_record, my_win_record, chances
    global last_bandit, total_reward,last_result
    if obs.step == 0 or obs.step ==1:
        print(obs)
    if obs.step == 0:        
        my_win_record = [8] * conf.banditCount
        my_loss_record = [1] * conf.banditCount
        my_chances = [8/9] * conf.banditCount
        chances = [0] * conf.banditCount
        bandit = random.randrange(conf.banditCount)
        last_bandit = bandit
        return bandit
    if obs.reward > total_reward:
        total_reward += 1
        my_win_record[last_bandit] += 1
        my_chances[last_bandit] = my_win_record[last_bandit] / (my_win_record[last_bandit] + my_loss_record[last_bandit])
        bandit = last_bandit        
    else:
        my_loss_record[last_bandit] += 1
        my_chances[last_bandit] = my_win_record[last_bandit] / (my_win_record[last_bandit] + my_loss_record[last_bandit])
        for i in range(conf.banditCount):
            chances[i] = my_chances[i] + 0.3/(my_win_record[i] + my_loss_record[i] + 1)
        bandit = chances.index(max(chances))        
        last_bandit = bandit
    #print(my_chances)
    #print(bandit)
    return bandit

In [ ]:
%%writefile ag_followHim.py

import numpy as np

decay_rate = 0.97
n_rounds = 2000
bandit_count = 100

total_reward = None 
last_bandit = None
his_hits = None
his_record = None
my_record = None
my_hits = None
wins = None
losses = None
bandits_record = None
record_index = None
x1 = None
x2 = None

sp = np.linspace(0,1,1000)
spp = 1 - sp
def probab(n_ones,n_zeros):
    global sp, spp
    cdfBeta = np.cumsum(sp**n_ones * spp **n_zeros )
    place = np.argmin(np.abs(cdfBeta - cdfBeta[999] / 2 ))
    return sp[place] + np.random.rand() * 0.0001

def decayed_probab(n_ones, n_zeros, his_n):
    global sp, spp
    ps = sp**n_ones * spp **n_zeros
    limit = int(1000 * decay_rate**(his_n + n_ones + n_zeros + 1 ) )+1
    cdfBeta = np.cumsum(ps[:limit] )
    place = np.argmin(np.abs(cdfBeta - cdfBeta[-1] / 2 ))
    return sp[place] + np.random.rand() * 0.0001

myProbabs = None
his_scores = None
his_bandit_rec = None
his_rec_index = None
def new_bandit(step):
    global myProbabs
    n_lookback = 49
    start = step - n_lookback if step >= 49 else 0
    his_last_moves = his_record[start:step]
    his_last_bandit = his_last_moves[-1]
    n_last_move = (his_last_moves == his_last_bandit).sum()
    
    myProbabs[last_bandit] = decayed_probab(wins[last_bandit],losses[last_bandit],his_hits[last_bandit])
    myProbabs[his_last_bandit] = decayed_probab(wins[his_last_bandit],losses[his_last_bandit],his_hits[his_last_bandit])
    
    if n_last_move > 1 and my_hits[his_last_bandit] <= 2:
        return his_last_bandit
    
    if n_last_move == 1 and his_hits[his_last_bandit] > 1 :
        myProbabs[his_last_bandit] -= .25
    


    exploratio_score = np.zeros(bandit_count)
    exploratio_score[his_hits == 0] = -0.25
    exploratio_score[his_hits == 1] = -0.5
    exploratio_score += 1 /(my_hits + 1)
    exploratio_score[my_hits>45] = -2
    scores = myProbabs + 0.1 * exploratio_score
    winner = int(np.argmax(scores))
    return winner

def agent(obs, conf):
    global bandits_record, my_record, my_hits, his_hits, his_record, myProbabs, his_scores
    global last_bandit, total_reward,  record_index, wins, losses, his_rec_index, his_bandit_rec
    if obs.step == 0:        
        total_reward = 0 
        his_record = np.zeros(n_rounds, 'int')
        his_hits = np.zeros(conf.banditCount, 'int')
        his_scores = np.zeros(bandit_count) + 0.5
        his_bandit_rec = np.ones([bandit_count, 600], 'int') * -1
        his_rec_index = np.zeros(bandit_count, 'int')
        
        myProbabs = np.random.rand(bandit_count)* 0.001 + 0.5
        my_record = np.zeros(n_rounds, 'int')
        my_hits = np.zeros(conf.banditCount, 'int')
        bandits_record = np.zeros([conf.banditCount, 600], 'int')
        record_index = np.zeros(conf.banditCount,'int')
        wins = np.zeros(conf.banditCount, 'int')
        losses = np.zeros(conf.banditCount, 'int')

        bandit = np.random.randint(conf.banditCount)
        last_bandit = bandit
        return bandit
    if obs.lastActions[0] == last_bandit:
        his_action = obs.lastActions[1]
    else:
        his_action = obs.lastActions[0]
    his_record[obs.step-1] = his_action
    his_hits[his_action] += 1
    his_bandit_rec[his_action,his_rec_index[his_action]] = obs.step
    his_rec_index[his_action] +=1
    my_hits[last_bandit] += 1
    my_record[obs.step-1] = last_bandit
    if obs.reward > total_reward:
        total_reward += 1
        bandits_record[last_bandit, record_index[last_bandit]] = 1
        wins[last_bandit] += 1
    else:
        bandits_record[last_bandit, record_index[last_bandit]] = 0
        losses[last_bandit] +=1
    record_index[last_bandit] += 1
    bandit = int(new_bandit(obs.step))
    last_bandit = bandit 
    return bandit


In [ ]:
agent1 = "myBestAgentInLB.py"
n_rounds = 10
agents = ["uniform.py", "random.py", "ucb_agent.py", "greedyMemory.py", "probabiliticRounds.py", "recentEvents.py", 'ag_followHim.py' ]

wins1 = [0] * len(agents)
wins2 = [0] * len(agents)
sum1 = [0] * len(agents)
sum2 = [0] * len(agents)
for agent in agents:
    print("\t", agent ,end='')
print()
for i in range(n_rounds):
    print(i+1, ":\t\t" ,end='')
    for j, agent in enumerate(agents):
        mygame = env.run([agent1, agent])
        #print(agent1, ':' , mygame[1999][0].observation.reward, end=" ")
        #print(agent2, ':' , mygame[1999][1].observation.reward, ,end='\t')
        #print(mygame[0][0].observation)
        #print(mygame[1][0].observation)
        sum1[j] += mygame[1999][0].observation.reward
        sum2[j] += mygame[1999][1].observation.reward
        if mygame[1999][0].observation.reward > mygame[1999][1].observation.reward:
            wins1[j] +=1
        if mygame[1999][0].observation.reward < mygame[1999][1].observation.reward:
            wins2[j] +=1
        print(wins1[j],wins2[j], end='\t\t')
    print()

for j, agent in enumerate(agents):
    print(f'{agent:<22}',wins1[j],wins2[j] ,sum1[j]/ n_rounds ,sum2[j] / n_rounds, sum1[j]/sum2[j], sep="\t")
